# Import Libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import scipy.stats as stat
import pickle
import statsmodels.api as sm
from statsmodels.othermod.betareg import BetaModel
from sklearn.linear_model import LogisticRegression

# Expected Loss

In [4]:
loan_data_inputs_train = pd.read_csv('loan_data_inputs_train.csv')
loan_data_inputs_test = pd.read_csv('loan_data_inputs_test.csv')
feature_pd = pd.read_csv('feature_pd.csv')
feature_lgd = pd.read_csv('feature_lgd.csv')
feature_ead = pd.read_csv('feature_ead.csv')
inputs = pd.concat([loan_data_inputs_train, loan_data_inputs_test], axis=0)

In [5]:
feature_pd = feature_pd.to_numpy()
feature_pd = feature_pd.flatten().tolist()

feature_lgd = feature_lgd.to_numpy()
feature_lgd = feature_lgd.flatten().tolist()

feature_ead = feature_ead.to_numpy()
feature_ead = feature_ead.flatten().tolist()

In [6]:
inputs_lgd = inputs[feature_lgd]
# Here we take only the accounts that were charged-off (written-off).

In [7]:
inputs_lgd.info(max_cols = 200)

<class 'pandas.core.frame.DataFrame'>
Index: 390077 entries, 0 to 78015
Data columns (total 46 columns):
 #   Column                                 Non-Null Count   Dtype  
---  ------                                 --------------   -----  
 0   grade_woe                              390077 non-null  float64
 1   home_ownership_woe                     390077 non-null  float64
 2   addr_state_woe                         390077 non-null  float64
 3   loan_amnt_scaled                       390077 non-null  float64
 4   int_rate_scaled                        390077 non-null  float64
 5   installment_scaled                     390077 non-null  float64
 6   annual_inc_scaled                      390077 non-null  float64
 7   fico_range_high_scaled                 390077 non-null  float64
 8   inq_last_6mths_scaled                  390077 non-null  float64
 9   mths_since_last_delinq_scaled          390077 non-null  float64
 10  mths_since_last_record_scaled          390077 non-null  float6

In [8]:
lgd_model = pickle.load(open('lgd_model.sav', 'rb'))
inputs_lgd = sm.add_constant(inputs_lgd)
inputs_lgd['recovery_rate'] = lgd_model.predict(inputs_lgd)


In [9]:
indices_alignes = inputs.index.equals(inputs_lgd.index)
print(f"Index matching : {indices_alignes}")

Index matching : True


In [10]:
inputs['recovery_rate'] = inputs_lgd['recovery_rate']

In [11]:
inputs['recovery_rate'] = np.where(inputs['recovery_rate'] < 0, 0, inputs['recovery_rate'])
inputs['recovery_rate'] = np.where(inputs['recovery_rate'] > 1, 1, inputs['recovery_rate'])
# We set estimated recovery rates that are greater than 1 to 1 and  estimated recovery rates that are less than 0 to 0.

In [12]:
inputs['LGD'] = 1 - inputs['recovery_rate']
inputs.head()
# We calculate estimated LGD. Estimated LGD equals 1 - estimated recovery rate.

,Unnamed: 0,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,tot_hi_cred_lim_scaled,total_bal_ex_mort_scaled,total_bc_limit_scaled,total_il_high_credit_limit_scaled,emp_length_int_scaled,mths_since_earliest_cr_line_scaled,term_int_scaled,mths_since_issue_d_scaled,recovery_rate,LGD
0,99617,110773804,4650.0,4650.0,4650.00,36 months,9.44,148.83,B,B1,...,0.001060,0.002136,0.016876,0.000000,0.0,0.278705,0.0,0.047619,0.023564,0.976436
1,57816,291118,10000.0,10000.0,7657.04,36 months,10.39,324.51,B,B4,...,0.000000,0.000000,0.000000,0.000000,0.4,0.222767,0.0,0.928571,0.016281,0.983719
2,47489,103127561,2000.0,2000.0,2000.00,36 months,14.99,69.33,C,C4,...,0.003000,0.006153,0.027694,0.008837,1.0,0.416094,0.0,0.071429,0.025227,0.974773
3,81869,112145963,40000.0,40000.0,40000.00,36 months,12.62,1340.45,C,C1,...,0.037216,0.039478,0.178278,0.066742,1.0,0.364082,0.0,0.039683,0.005925,0.994075
4,224813,19587473,20000.0,20000.0,20000.00,36 months,9.17,637.58,B,B1,...,0.012880,0.000955,0.014929,0.000000,1.0,0.544652,0.0,0.325397,0.007992,0.992008


In [13]:
inputs['LGD'].describe()
# Shows some descriptive statisics for the values of a column.

count    390077.000000
mean          0.971526
std           0.039744
min           0.000207
25%           0.964272
50%           0.978649
75%           0.991246
max           1.000000
Name: LGD, dtype: float64

In [14]:
from sklearn import linear_model
import scipy.stats as stat

class LinearRegression(linear_model.LinearRegression):
    """
    LinearRegression class after sklearn's, but calculate t-statistics
    and p-values for model coefficients (betas).
    Additional attributes available after .fit()
    are `t` and `p` which are of the shape (n_features,)
    This class sets the intercept to 0 by default, since usually we include it
    in X.
    """
    
    def __init__(self, fit_intercept=True, copy_X=True, n_jobs=None, positive=False):
        # Initialisation correcte du constructeur de la classe parente
        super(LinearRegression, self).__init__(fit_intercept=fit_intercept,
                                               copy_X=copy_X,
                                               n_jobs=n_jobs, positive=positive)
    
    def fit(self, X, y):
        # Appel à la méthode fit de la classe parente
        super(LinearRegression, self).fit(X, y)

        # Calcul du SSE (Sum of Squared Errors)
        sse = np.sum((self.predict(X) - y) ** 2, axis=0) / float(X.shape[0] - X.shape[1])

        # Calcul de l'erreur standard (SE)
        se = np.array([np.sqrt(np.diagonal(sse * np.linalg.inv(np.dot(X.T, X))))])

        # Calcul du t-statistic pour chaque coefficient
        self.t = self.coef_ / se

        # Calcul des p-values pour chaque coefficient
        self.p = np.squeeze(2 * (1 - stat.t.cdf(np.abs(self.t), X.shape[0] - X.shape[1])))
        
        return self

In [15]:
inputs_ead = inputs[feature_ead]
# Here we take only the accounts that were charged-off (written-off).
ead_model = pickle.load(open('ead_model.sav', 'rb'))
inputs_ead['CCF'] = ead_model.predict(inputs_ead)
# We apply the EAD model to calculate estimated credit conversion factor.

C:\Users\Pierre Pro\AppData\Local\Temp\ipykernel_14168\3984565163.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inputs_ead['CCF'] = ead_model.predict(inputs_ead)


In [16]:
indices_alignes = inputs.index.equals(inputs_ead.index)
print(f"Index matching : {indices_alignes}")

Index matching : True


In [17]:
inputs['CCF'] = inputs_ead['CCF']

In [18]:
inputs['CCF'] = np.where(inputs['CCF'] < 0, 0, inputs['CCF'])
inputs['CCF'] = np.where(inputs['CCF'] > 1, 1, inputs['CCF'])
# We set estimated CCF that are greater than 1 to 1 and  estimated CCF that are less than 0 to 0.

In [19]:
inputs['EAD'] = inputs['CCF'] * inputs['funded_amnt']
# We calculate estimated EAD. Estimated EAD equals estimated CCF multiplied by funded amount.

In [20]:
inputs['EAD'].describe()
# Shows some descriptive statisics for the values of a column.

count    390077.000000
mean       5145.281317
std        5707.826893
min           0.000000
25%        2291.891174
50%        3417.099884
75%        5739.870701
max       40000.000000
Name: EAD, dtype: float64

In [21]:
inputs.head()

,Unnamed: 0,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,total_bc_limit_scaled,total_il_high_credit_limit_scaled,emp_length_int_scaled,mths_since_earliest_cr_line_scaled,term_int_scaled,mths_since_issue_d_scaled,recovery_rate,LGD,CCF,EAD
0,99617,110773804,4650.0,4650.0,4650.00,36 months,9.44,148.83,B,B1,...,0.016876,0.000000,0.0,0.278705,0.0,0.047619,0.023564,0.976436,0.574358,2670.767000
1,57816,291118,10000.0,10000.0,7657.04,36 months,10.39,324.51,B,B4,...,0.000000,0.000000,0.4,0.222767,0.0,0.928571,0.016281,0.983719,0.281416,2814.164019
2,47489,103127561,2000.0,2000.0,2000.00,36 months,14.99,69.33,C,C4,...,0.027694,0.008837,1.0,0.416094,0.0,0.071429,0.025227,0.974773,0.662955,1325.909472
3,81869,112145963,40000.0,40000.0,40000.00,36 months,12.62,1340.45,C,C1,...,0.178278,0.066742,1.0,0.364082,0.0,0.039683,0.005925,0.994075,0.300269,12010.751518
4,224813,19587473,20000.0,20000.0,20000.00,36 months,9.17,637.58,B,B1,...,0.014929,0.000000,1.0,0.544652,0.0,0.325397,0.007992,0.992008,0.035735,714.709407


In [25]:
import pickle
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
import scipy.stats as stat

class LogisticRegression_with_p_values2:
    
    def __init__(self, *args, **kwargs):
        if 'max_iter' not in kwargs:
            kwargs['max_iter'] = 1000
        self.model = LogisticRegression(*args, **kwargs)

    def fit(self, X, y):
        # Ajuster le modèle
        self.model.fit(X, y.values.ravel())  # Utilisez .ravel() pour aplatir la dataframe en vecteur
        
        # Convertir les données pour statsmodels
        X_sm = sm.add_constant(X)
        model_sm = sm.Logit(y, X_sm)
        result = model_sm.fit()
        
        # Obtenir les p-values
        self.coef_ = self.model.coef_
        self.intercept_ = self.model.intercept_
        self.p_values = result.pvalues[1:]  # Exclure l'interception

In [27]:
inputs_pd = inputs[feature_pd]
# Here we keep only the variables we need for the model.
pd_model = pickle.load(open('pd_model.sav', 'rb'))

In [29]:
inputs_pd.shape

(390077, 46)

In [30]:
inputs_pd['PD'] = pd_model.model.predict_proba(inputs_pd)[: ][: , 1]
# We apply the PD model to caclulate estimated default probabilities.

C:\Users\Pierre Pro\AppData\Local\Temp\ipykernel_14168\4217429293.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inputs_pd['PD'] = pd_model.model.predict_proba(inputs_pd)[: ][: , 1]


In [31]:
indices_alignes = inputs.index.equals(inputs_pd.index)
print(f"Index matching : {indices_alignes}")

Index matching : True


In [40]:
inputs['PD'] = inputs_pd['PD']

In [42]:
inputs['PD'].head()

0    0.001263
1    0.014664
2    0.002879
3    0.072349
4    0.000167
Name: PD, dtype: float64

In [43]:
inputs['PD'].describe()
# Shows some descriptive statisics for the values of a column.

count    3.900770e+05
mean     1.595580e-01
std      3.371605e-01
min      4.016297e-14
25%      5.178669e-04
50%      6.634522e-03
75%      4.352194e-02
max      1.000000e+00
Name: PD, dtype: float64

In [54]:
pd.options.display.max_rows = None
inputs.head()

,Unnamed: 0,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,total_il_high_credit_limit_scaled,emp_length_int_scaled,mths_since_earliest_cr_line_scaled,term_int_scaled,mths_since_issue_d_scaled,recovery_rate,LGD,CCF,EAD,PD
0,99617,110773804,4650.0,4650.0,4650.00,36 months,9.44,148.83,B,B1,...,0.000000,0.0,0.278705,0.0,0.047619,0.023564,0.976436,0.574358,2670.767000,0.001263
1,57816,291118,10000.0,10000.0,7657.04,36 months,10.39,324.51,B,B4,...,0.000000,0.4,0.222767,0.0,0.928571,0.016281,0.983719,0.281416,2814.164019,0.014664
2,47489,103127561,2000.0,2000.0,2000.00,36 months,14.99,69.33,C,C4,...,0.008837,1.0,0.416094,0.0,0.071429,0.025227,0.974773,0.662955,1325.909472,0.002879
3,81869,112145963,40000.0,40000.0,40000.00,36 months,12.62,1340.45,C,C1,...,0.066742,1.0,0.364082,0.0,0.039683,0.005925,0.994075,0.300269,12010.751518,0.072349
4,224813,19587473,20000.0,20000.0,20000.00,36 months,9.17,637.58,B,B1,...,0.000000,1.0,0.544652,0.0,0.325397,0.007992,0.992008,0.035735,714.709407,0.000167


In [56]:
inputs['EL'] = inputs['PD'] * inputs['LGD'] * inputs['EAD']
# We calculate Expected Loss. EL = PD * LGD * EAD.

In [57]:
inputs['EL'].describe()
# Shows some descriptive statisics for the values of a column.

count    390077.000000
mean       1573.165561
std        4442.628499
min           0.000000
25%           0.616544
50%          19.049796
75%         237.532174
max       39651.272111
Name: EL, dtype: float64

In [58]:
inputs[['funded_amnt', 'PD', 'LGD', 'EAD', 'EL']].head()

,funded_amnt,PD,LGD,EAD,EL
0,4650.0,0.001263,0.976436,2670.767000,3.292475
1,10000.0,0.014664,0.983719,2814.164019,40.594904
2,2000.0,0.002879,0.974773,1325.909472,3.721409
3,40000.0,0.072349,0.994075,12010.751518,863.815369
4,20000.0,0.000167,0.992008,714.709407,0.118714


In [59]:
inputs = inputs.loc[:, ~inputs.columns.duplicated()]
inputs['funded_amnt'].describe()

count    390077.000000
mean      14763.861686
std        8882.078068
min         500.000000
25%        8000.000000
50%       12600.000000
75%       20000.000000
max       40000.000000
Name: funded_amnt, dtype: float64

In [60]:
inputs['EL'].sum()
# Total Expected Loss for all loans.

613655702.723173

In [61]:
inputs['funded_amnt'].sum()
# Total funded amount for all loans.

5759042875.0

In [62]:
inputs['EL'].sum() / inputs['funded_amnt'].sum()
# Total Expected Loss as a proportion of total funded amount for all loans.
####
####
####
# THE END.

0.10655515439675782